In [1]:
!pip install -U transformers accelerate bitsandbytes


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 26.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.4
    Uninstalling tokenizers-0.21.4:
      Successfully uninstalled tokenizers-0.21.4
  Attempting uninstall: transformers
    Found existing installation: transformers 4.55.4
    Uninstalling transformers-4.55.4:
      Successfully uninstalled transformers-4.55.4


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
import torch

MODEL_ID = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# Pick ONE of these:
# ---- 8-bit INT8 ----
# bnb_config = BitsAndBytesConfig(load_in_8bit=True)

# ---- 4-bit NF4 (QLoRA-style) ----
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",        # nf4 or fp4
    bnb_4bit_use_double_quant=True,   # second quantization for better compression
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto"  # puts layers on your GPU(s) automatically
)

# Simple generation
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")
prompt = "Explain quantization in neural networks in one paragraph."
out = pipe(prompt, max_new_tokens=120, do_sample=True, temperature=0.7)
print(out[0]["generated_text"])

# Save "quantized" model (note: config + metadata; needs bitsandbytes to reload)
save_dir = "tinyllama-1.1b-chat-4bit"
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)
print(f"Saved to: {save_dir}")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cpu


Explain quantization in neural networks in one paragraph. (100 words)
Saved to: tinyllama-1.1b-chat-4bit


In [3]:
prompt = "Explain quantization in simple words."
output = pipe(prompt, max_new_tokens=100, do_sample=True, temperature=0.7)

print("\n--- Model Output ---\n")
print(output[0]["generated_text"])



--- Model Output ---

Explain quantization in simple words. 5. A/D Conversion: A/D (analog-to-digital) conversion is a process of converting the digital output from a microcontroller to an analog output. It is an essential component of the signal processing chain, and the microcontroller performs this conversion. 6. Timer: A timer is a clock-based system that controls the timing of other processes. In the analog-to-digital conversion, a timer is used to regulate the ADC clock
